## Output Parsers
Output parsers parse the output expecting the output to be in a specified format like,
- Text/String
- Specified schema
- Comma Separated output
- Data Time etc.

In [ ]:
!pip install langchain
!pip install langchain-core
!pip install langchain_community
!pip install langchain_openai

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI

dictionary_template = PromptTemplate(
    input_variables = ["word"],
    template="Give one line definition of {word}. Response should not contain the word itself only response."
)

llm = ChatOpenAI()

chain = dictionary_template | llm

chain.invoke({"word":"Python"})

AIMessage(content='Python is a high-level programming language known for its readability and simplicity.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 24, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bslhe7Rj1ncZEkySVwpMmK6mwf3CN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c18906b-71a0-425b-af3c-322f0da28a28-0', usage_metadata={'input_tokens': 24, 'output_tokens': 14, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

##  StrOutputParser
Parses text or string output content

In [ ]:
from langchain import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

output_parser =  StrOutputParser()

dictionary_template = PromptTemplate(
    input_variables = ["word"],
    template="Give one line definition of {word}. Response should not contain the word itself only response."
)

llm = ChatOpenAI()

chain = dictionary_template | llm | output_parser

chain.invoke({"word":"Python"})

'Python is a high-level programming language that is versatile and easy to read.'

## StructuredOutputParser
Structured output can follow a specified structure for output

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

response_schemas = [
    ResponseSchema(name="word", description="word to be defined"),
    ResponseSchema(name="definition", description="definition of word"),
    ResponseSchema(name="example", description="example of usage of word"),
    ResponseSchema(name="discovery", description="who discovered it along with date of discovery and place"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"word": string  // word to be defined
	"definition": string  // definition of word
	"example": string  // example of usage of word
	"discovery": string  // who discovered it along with date of discovery and place
}
```


In [ ]:
prompt = PromptTemplate(
    input_variables=["word"],
    template="Give one line definition of {word}. Response should not contain the word itself only response. Give and Example also and when it was discovered. {format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | chat_model | output_parser
chain.invoke("Anaconda")

{'word': 'Anaconda',
 'definition': 'A large non-venomous snake found in South America with the ability to constrict its prey',
 'example': 'The anaconda wrapped itself around the deer to suffocate it before consuming it whole.',
 'discovery': 'Discovered by European explorers in South America during the early 16th century.'}

In [ ]:
prompt = PromptTemplate(
    input_variables=["word"],
    template="provide information about {word} \n{format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | chat_model | output_parser
chain.invoke("Anaconda")

{'word': 'Anaconda',
 'definition': 'Anaconda is a free and open-source distribution of the Python and R programming languages for data science and machine learning related applications.',
 'example': 'I use Anaconda to manage my Python packages and environments for my data analysis projects.',
 'discovery': 'Anaconda was developed by Continuum Analytics, which later merged with Anaconda Inc. It was initially released in 2012.'}

## CommaSeparatedListOutputParser

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

# gives default format instruction template
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    input_variables=["topic"],
    template="Give the user five types of {topic}.\n{format_instructions}",
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | llm | output_parser

In [ ]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [ ]:
print(chain.invoke({"topic": "programming languages"}))

['Java', 'Python', 'JavaScript', 'C++', 'Ruby']


## DateTime Parser

In [ ]:
from langchain.output_parsers import DatetimeOutputParser, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI()

# gives default format instruction template
output_parser = DatetimeOutputParser()

format_instructions = output_parser.get_format_instructions()
template = """Answer these questions
{question}

{format_instructions}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": format_instructions}
)



In [ ]:
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={'format_instructions': "Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 298-09-06T21:21:07.380013Z, 1870-10-30T23:44:54.731636Z, 1082-03-12T01:57:14.920685Z\n\nReturn ONLY this string, no other words!"}, template='Answer these questions\n{question}\n\n{format_instructions}\n')

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"question": "What is the date and time now?"})

datetime.datetime(2022, 6, 21, 14, 30, 45, 123456)

In [ ]:
output = chain.invoke({"question": "When did the first man land on the moon?"})
print(output)

1969-07-20 20:17:40
